In [1]:
import pandas as pd
import numpy as np
import pyunpack
import math
import json

from models import GatedLinearUnit
from models import GateAddNormNetwork
from models import GatedResidualNetwork 
from models import ScaledDotProductAttention
from models import InterpretableMultiHeadAttention
from models import VariableSelectionNetwork

from data_formatters.m5_faster import M5Formatter
from datasets.M5dataset import TFTDataset
from data_formatters.base import GenericDataFormatter
from data_formatters.base import DataTypes, InputTypes

from quantile_loss import QuantileLossCalculator
from quantile_loss import NormalizedQuantileLossCalculator

import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch import nn

import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping
from argparse import ArgumentParser

import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

In [1]:
from utils.utils import get_single_col_by_input_type
from utils.utils import extract_cols_from_data_type

from data_formatters.base import DataTypes, InputTypes

In [2]:
import pandas as pd
import modin.pandas as mpd
import dask.dataframe as dd
import numpy as np
import cudf
import dask_cudf
import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

## Column Definition

In [3]:
column_definitions = [
  ('id', DataTypes.CATEGORICAL, InputTypes.ID),
  ('sales', DataTypes.REAL_VALUED, InputTypes.TARGET),

  ('d', DataTypes.CATEGORICAL, InputTypes.TIME),
  ('event_name_2', DataTypes.CATEGORICAL, InputTypes.OBSERVED_INPUT),
  ('event_type_2', DataTypes.CATEGORICAL, InputTypes.OBSERVED_INPUT),
  ('event_name_1', DataTypes.CATEGORICAL, InputTypes.OBSERVED_INPUT),
  ('event_type_1', DataTypes.CATEGORICAL, InputTypes.OBSERVED_INPUT),
  ('snap_WI', DataTypes.CATEGORICAL, InputTypes.KNOWN_INPUT),
  ('snap_CA', DataTypes.CATEGORICAL, InputTypes.KNOWN_INPUT),
  ('snap_TX', DataTypes.CATEGORICAL, InputTypes.KNOWN_INPUT),

  ('price_momentum', DataTypes.REAL_VALUED, InputTypes.KNOWN_INPUT),
  ('price_momentum_m', DataTypes.REAL_VALUED, InputTypes.KNOWN_INPUT),
  ('price_momentum_y', DataTypes.REAL_VALUED, InputTypes.KNOWN_INPUT),

  ('price_max', DataTypes.REAL_VALUED, InputTypes.KNOWN_INPUT),
  ('price_min', DataTypes.REAL_VALUED, InputTypes.KNOWN_INPUT),
  ('price_std', DataTypes.REAL_VALUED, InputTypes.KNOWN_INPUT),
  ('price_mean', DataTypes.REAL_VALUED, InputTypes.KNOWN_INPUT),
  ('price_norm', DataTypes.REAL_VALUED, InputTypes.KNOWN_INPUT),
  ('price_nunique', DataTypes.REAL_VALUED, InputTypes.KNOWN_INPUT),
  ('sell_price', DataTypes.REAL_VALUED, InputTypes.KNOWN_INPUT),

  ('tm_d', DataTypes.CATEGORICAL, InputTypes.KNOWN_INPUT),
  ('tm_w', DataTypes.CATEGORICAL, InputTypes.KNOWN_INPUT),
  ('tm_m', DataTypes.CATEGORICAL, InputTypes.KNOWN_INPUT),
  ('tm_y', DataTypes.CATEGORICAL, InputTypes.KNOWN_INPUT),
  ('tm_wm', DataTypes.CATEGORICAL, InputTypes.KNOWN_INPUT),
  ('tm_dw', DataTypes.CATEGORICAL, InputTypes.KNOWN_INPUT),
  ('tm_w_end', DataTypes.CATEGORICAL, InputTypes.KNOWN_INPUT),

  ('snap_TX', DataTypes.REAL_VALUED, InputTypes.OBSERVED_INPUT),
  ('snap_CA', DataTypes.REAL_VALUED, InputTypes.OBSERVED_INPUT),
  ('snap_WI', DataTypes.REAL_VALUED, InputTypes.OBSERVED_INPUT),

  ('item_nunique', DataTypes.CATEGORICAL, InputTypes.STATIC_INPUT),
  ('release', DataTypes.CATEGORICAL, InputTypes.STATIC_INPUT), 
  ('item_id', DataTypes.CATEGORICAL, InputTypes.STATIC_INPUT), 
  ('dept_id', DataTypes.CATEGORICAL, InputTypes.STATIC_INPUT),
  ('cat_id', DataTypes.CATEGORICAL, InputTypes.STATIC_INPUT),
  ('store_id', DataTypes.CATEGORICAL, InputTypes.STATIC_INPUT),
  ('state_id', DataTypes.CATEGORICAL, InputTypes.STATIC_INPUT),
]

## Creating Datasets

In [4]:
m5_data = pd.read_pickle('/container/home/millenium/Storage/Daniel/github/kaggle_competitions/M5_Forecasting_Accuracy/data/full_data_no_nan.pkl')
m5_data = m5_data[m5_data.d <= 1913]
m5_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46027957 entries, 0 to 46027956
Data columns (total 34 columns):
 #   Column            Dtype   
---  ------            -----   
 0   id                category
 1   item_id           category
 2   dept_id           category
 3   cat_id            category
 4   store_id          category
 5   state_id          category
 6   d                 int16   
 7   sales             float64 
 8   release           int16   
 9   sell_price        float16 
 10  price_max         float16 
 11  price_min         float16 
 12  price_std         float16 
 13  price_mean        float16 
 14  price_norm        float16 
 15  price_nunique     float16 
 16  item_nunique      int16   
 17  price_momentum    float16 
 18  price_momentum_m  float16 
 19  price_momentum_y  float16 
 20  event_name_1      category
 21  event_type_1      category
 22  event_name_2      category
 23  event_type_2      category
 24  snap_CA           category
 25  snap_TX         

In [6]:
m5_data_d = dd.from_pandas(m5_data, npartitions = 128)

In [8]:
m5_data_d = m5_data_d.set_index('id').as_ordered()

TypeError: Categorical is not ordered for operation max
you can use .as_ordered() to change the Categorical to an ordered one
